## Albania isolates with reference genome set

* C. Loiseau et al., “An African origin for Mycobacterium bovis,” Evol. Med. Public Heal., pp. 49–59, 2020.

In [1]:
import sys,os,shutil,subprocess
import glob
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
from Bio import SeqIO
sys.path.append('pathogenie')
from pathogenie import tools, aligners, app, trees

In [2]:
master = pd.read_csv('../mbovis_sra_master.csv')

In [ ]:
reload(app)
args = {'threads':8, 'outdir': '../test_results', 'labelsep':'-',
        'input':['/storage/btbgenie/mbovis_sra/',                 
                 '/storage/btbgenie/albania/'],
        'reference': None, 'overwrite':False}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

## Create tree labels with meta data

In [ ]:
reload(trees)
cols = ['Run','COUNTRY_ISOLATION','CLONAL_COMPLEX','LibraryLayout','Host','ReleaseDate','GENOME_COVERAGE','filename']
df = W.fastq_table
df = df.merge(master,left_on='sample',right_on='Run',how='left').drop_duplicates('bam_file')
#print (df.loc[0])
df.at[0:2,'COUNTRY_ISOLATION']='Albania'
labelmap = dict(zip(df.bam_file,df.COUNTRY_ISOLATION))
labelmap

colors = {'Eu1': 'Pink', 'Eu2': 'Green','Af1':'Orange','Af2':'Blue','Unknown2':'gray',np.nan:'gray'}
df['color'] = df.CLONAL_COMPLEX.apply(lambda x: colors[x],1)
colormap = dict(zip(df.bam_file,df.color))
t = trees.create_tree('../test_results/RAxML_bipartitions.variants', 'ref', labelmap, colormap)
t.render("%%inline",w=400,dpi=100)